In [2]:
import pandas as pd
import numpy as np
from glob import glob

In [5]:
#get list of all data files
data_files = sorted(glob('data/*.csv'))
data_files

['data\\202103-tripdata_v1.csv',
 'data\\202104-tripdata_v1.csv',
 'data\\202105-tripdata_v1.csv',
 'data\\202106-tripdata_v1.csv',
 'data\\202107-tripdata_v1.csv',
 'data\\202108-tripdata_v1.csv',
 'data\\202109-tripdata_v1.csv',
 'data\\202110-tripdata_v1.csv',
 'data\\202111-tripdata_v1.csv',
 'data\\202112-tripdata_v1.csv',
 'data\\202201-tripdata_v1.csv',
 'data\\202202-tripdata_v1.csv',
 'data\\202203-tripdata_v1.csv']

In [6]:
data = pd.concat((pd.read_csv(file) for file in data_files), ignore_index=True)

C:\Users\Azhar\miniconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\Azhar\miniconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [8]:
data.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'end_station_name', 'member_casual',
       'ride_length', 'day_of_week', 'start_station_id', 'end_station_id',
       'Unnamed: 11', 'station_id', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

In [9]:
df = data.drop(['Unnamed: 11', 'station_id', 'Unnamed: 12', 'Unnamed: 13'], axis=1)
df.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day_of_week,start_station_id,end_station_id
0,CFA86D4455AA1030,classic_bike,3/16/2021 8:32,3/16/2021 8:36,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,casual,0:04:04,3,15651.0,13266.0
1,30D9DC61227D1AF3,classic_bike,3/28/2021 1:26,3/28/2021 1:36,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,casual,0:10:27,1,15651.0,18017.0
2,846D87A15682A284,classic_bike,3/11/2021 21:17,3/11/2021 21:33,Shields Ave & 28th Pl,Halsted St & 35th St,casual,0:16:24,5,15443.0,13080.0
3,994D05AA75A168F2,classic_bike,3/11/2021 13:26,3/11/2021 13:55,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,casual,0:28:59,5,13080.0,13323.0
4,DF7464FBE92D8308,classic_bike,3/21/2021 9:09,3/21/2021 9:27,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,casual,0:17:56,1,525.0,8.0


In [10]:
df.shape

(5691916, 11)

# Data Cleaning

In [12]:
#Percentage of NAN Values 
missing_values = [(c, df[c].isna().mean()*100) for c in df]
missing_values = pd.DataFrame(missing_values, columns=["column_name", "percentage"])
missing_values

,column_name,percentage
0,ride_id,0.000000
1,rideable_type,0.000000
2,started_at,0.000000
3,ended_at,0.000000
4,start_station_name,11.648239
5,end_station_name,12.175654
6,member_casual,0.000000
7,ride_length,10.849088
8,day_of_week,10.849088
9,start_station_id,11.713806


In [14]:
df.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    663008
end_station_name      693028
member_casual              0
ride_length           617521
day_of_week           617521
start_station_id      666740
end_station_id        696806
dtype: int64

In [15]:
df.dropna(axis=0, inplace=True)

In [16]:
df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
end_station_name      0
member_casual         0
ride_length           0
day_of_week           0
start_station_id      0
end_station_id        0
dtype: int64

In [17]:
#identify duplicate rows
duplicateRows = df[df.duplicated()]
duplicateRows

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day_of_week,start_station_id,end_station_id


In [20]:
df.shape

(4166078, 11)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4166078 entries, 0 to 5691915
Data columns (total 11 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   ride_id             object
 1   rideable_type       object
 2   started_at          object
 3   ended_at            object
 4   start_station_name  object
 5   end_station_name    object
 6   member_casual       object
 7   ride_length         object
 8   day_of_week         object
 9   start_station_id    object
 10  end_station_id      object
dtypes: object(11)
memory usage: 381.4+ MB


In [31]:
# remove alphabets from station_id columns
df['start_station_id'] = df.start_station_id.replace('[^\d.]', '', regex=True).astype(float)

ValueError: could not convert string to float: 

In [19]:
df.to_csv('data/data_cleaned.csv', index=False)